In [2]:
import random
import requests
import pandas as pd
import json
import datetime as dt
import time
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from headers import headers_list
from data_skills import DATA_SKILLS
from skill_extraction import extract_skills, extract_ignore, extract_data_skills
from secrets import settings, api_keys

## JMLR

In [ ]:
def jmlr_scraper(engine):
    base_url = 'https://jmlr.org'
    url = base_url + '/papers/v22/'
    page = requests.get(url, headers=random.choice(headers_list))
    if page.status_code != 200:
        return
    soup = BeautifulSoup(page.content, 'html.parser')
    dls = soup.findAll('dl')
    # Get existing papers in database
    df_ex = pd.read_sql_query('select cj.id, cj.title from "ContentJMLR" cj', engine)
    ex_papers = df_ex['title'].unique().tolist()
    papers = []
    # Iterate through each paper
    has_new = False
    for dl in dls:
        title = dl.find('dt').get_text()
        if title in ex_papers:
            continue
        paper = {}
        dd = dl.find('dd')
        paper['title'] = title
        paper['authors'] = dd.get_text().split(';')[0].strip()
        paper['journal_num'] = dd.get_text().split(';')[-1].split('\n')[0].strip()
        for a in dd.findAll('a'):
            if a.get_text() == '(Machine Learning Open Source Software Paper)':
                continue
            href = a['href']
            if 'http' not in href:
                href = 'https://jmlr.org' + href
            paper[a.get_text()] = href
        # Get abstract of paper and extract skills
        output = get_abstract_skills(paper)
        if output is not None:
            paper['abstract'] = output[0]
            if len(output[1]) > 0:
                paper['skills'] = '; '.join(output[1])
                data_skills = extract_data_skills(output[1])
                if len(data_skills) > 0:
                    paper['data_skills'] = '; '.join(data_skills)
        papers.append(paper)
        has_new = True
    # Compile into dataframe if we have new papers
    if has_new:
        df = pd.DataFrame.from_dict(papers)
        df['id'] = df.index + max(df_ex['id']) + 1
        return df
    return None

def get_abstract_skills(paper):
    page = requests.get(paper['abs'], headers=random.choice(headers_list))
    if page.status_code != 200:
        return None
    soup = BeautifulSoup(page.content, 'html.parser')
    abstract = soup.find('p', class_='abstract').get_text().strip('\n')
    all_skills = extract_skills(paper['title'] + ' ' + abstract)
    keep_skills, _ = extract_ignore(all_skills)
    keep_skills.sort()
    return abstract, keep_skills

In [ ]:
engine = create_engine(settings['skills_db'])
df_jmlr = jmlr_scraper(engine)
df_jmlr.head()

In [ ]:
engine.dispose()

In [ ]:
df_jmlr.to_csv('database/jmlr.csv', index=False)

## Youtube

In [ ]:
def get_youtube_videos(skill, filter_time=None):
    base_url = 'https://www.youtube.com'
    # Dictionary for filtering search query
    sp_dict = {'this_year': 'EgQIBRAB', 'this_month': 'EgQIBBAB', 'this_week': 'EgQIAxAB', 'today': 'EgQIAhAB'}
    if filter_time not in sp_dict.keys():
        return None
    url = base_url + '/results'
    query = 'learn ' + skill
    params = {'search_query': query.replace(' ', '+')}
    # Default is no filter
    if filter_time is not None:
        params['sp'] = sp_dict[filter_time]
    page = requests.get(url, params=params, headers=random.choice(headers_list))
    if page.status_code != 200:
        print(page, page.reason)
        return None
    soup = BeautifulSoup(page.content, 'html.parser')
    json_text = str(soup.find_all('script')).split('var ytInitialData = ')[-1].split(';</script>')[0]
    res = json.loads(json_text)
    res = res['contents']['twoColumnSearchResultsRenderer']['primaryContents']['sectionListRenderer']['contents']
    video_list = []
    # Iterate through each video
    for contents in res:
        # Get only those with video
        if 'itemSectionRenderer' not in contents:
            continue
        contents = contents['itemSectionRenderer']['contents']
        for content in contents:
            # Ignore ads
            if 'videoRenderer' not in content:
                continue
            content = content['videoRenderer']
            title = get_text(content, 'title')
            if title is None:
                continue
            description = get_description(content)
            skills, data_skills = get_skills(title, description)
            published_year, published_month = get_published_date(content)
            video_list.append({
                'id': content['videoId'],
                'title': title,
                'channel': get_text(content, 'ownerText'),
                'published_year': published_year,
                'published_month': published_month,
                'length': get_length(content),
                'view_count': get_view_count(content),
                'url': get_url(content),
                'description': description,
                'skills': skills,
                'data_skills': data_skills
            })
    df = pd.DataFrame.from_dict(video_list)
    # df['length'] = pd.to_timedelta(df['length'])
    return df

def get_text(content, info):
    try:
        return ' '.join(t['text'] for t in content[info]['runs'])
    except:
        return None

def get_length(content):
    try:
        length = content['lengthText']['simpleText']
        length = length.split(':')
        if len(length) == 1:
            length.insert(0, '00')
        if len(length) == 2:
            length.insert(0, '00')
        return ':'.join(length)
    except:
        return None

def get_published_date(content):
    try:
        published_time = content['publishedTimeText']['simpleText']
        val = [int(s) for s in published_time.split() if s.isdigit()][0]
        current = dt.datetime.now()
        if 'year' in published_time:
            published = current - dt.timedelta(days=365.25*val)
        elif 'month' in published_time:
            published = current - dt.timedelta(days=30.436875*val)
        elif 'week' in published_time:
            published = current - dt.timedelta(weeks=val)
        elif 'day' in published_time:
            published = current - dt.timedelta(days=val)
        elif 'hour' in published_time:
            published = current - dt.timedelta(hours=val)
        elif 'minute' in published_time:
            published = current - dt.timedelta(minutes=val)
        elif 'second' in published_time:
            published = current - dt.timedelta(seconds=val)
        return published.year, published.month
    except:
        return None, None

def get_view_count(content):
    try:
        view_count = content['viewCountText']['simpleText']
        view_count = view_count.split(' views')[0].replace(',', '')
        return int(view_count)
    except:
        return None

def get_url(content):
    try:
        url = content['navigationEndpoint']['commandMetadata']['webCommandMetadata']['url']
        return base_url + url
    except:
        return None

def get_description(content):
    try:
        description = ' '.join([t['text'] for t in content['detailedMetadataSnippets'][0]['snippetText']['runs']])
        return description
    except:
        return None

def get_skills(title, description):
    context = title
    if description is not None:
        context = context + ' ' + description
    all_skills = extract_skills(context)
    # Ignore the Video skill as it is not relevant for Youtube
    if 'Video' in all_skills:
        all_skills.remove('Video')
    keep_skills, _ = extract_ignore(all_skills)
    keep_skills.sort()
    if len(keep_skills) > 0:
        data_skills = extract_data_skills(keep_skills)
        if len(data_skills) > 0:
            return '; '.join(keep_skills), '; '.join(data_skills)
        return '; '.join(keep_skills), None
    return None, None

In [ ]:
df_yt = pd.DataFrame()

for skill in DATA_SKILLS:
    try:
        df_temp = get_youtube_videos(skill, 'this_month')
        df_yt = df_yt.append(df_temp)
    except Exception as e:
        print('Error in scraping Youtube for {}'.format(skill), e)
    time.sleep(5)

df_yt.head()

In [ ]:
engine = create_engine(settings['skills_db2'])
df_yt = pd.read_csv('database/youtube.csv')
df_yt['length'] = df_yt['length'].apply(lambda x: x.split()[-1])
df_yt.head()

In [ ]:
df_yt.to_sql('ContentYoutube', engine, index=False, if_exists='replace')
engine.dispose()

In [ ]:
len(df_yt), len(df_yt[~df_yt['data_skills'].isna()])

## Medium

In [3]:
def medium_scraper(tag, date):
    base_url = 'https://medium.com/tag/{}/archive/'
    url = base_url.format(tag) + date.strftime('%Y/%m/%d')
    page = requests.get(url, headers=random.choice(headers_list))
    soup = BeautifulSoup(page.content, 'html.parser')
    # Pulls each card from the feed. Each card is a story or comment
    cards = soup.find_all('div', class_='streamItem streamItem--postPreview js-streamItem')
    card_list = []
    for card in cards:
        title = get_title(card)
        subtitle = get_subtitle(card)
        claps = get_claps(card)
        if title is None or is_comment(card) or claps is None:
            continue
        if claps < 100:
            continue
        skills, data_skills = get_skills(title, subtitle)
        card_list.append({
            'id': get_id(card),
            'title': title,
            'subtitle': subtitle,
            'author': get_author(card),
            'publication': get_publication(card),
            'published_date': date,
            'read_time_mins': get_read_time(card),
            'claps': claps,
            'url': get_url(card),
            'skills': skills,
            'data_skills': data_skills,
        })
    df = pd.DataFrame.from_dict(card_list)
    return df

def get_id(card):
    id_ = card.find('div', class_='postArticle postArticle--short js-postArticle js-trackPostPresentation js-trackPostScrolls')
    if id_ is not None:
        return id_['data-post-id']
    return id_

def get_title(card):
    # Different combination of classes possible for titles
    combinations = [('h3', 'graf graf--h3 graf-after--figure graf--title'),
                    ('h3', 'graf graf--h3 graf-after--figure graf--trailing graf--title'),
                    ('h4', 'graf graf--h4 graf--leading'),
                    ('h3', 'graf graf--h3 graf--leading graf--title'),
                    ('p', 'graf graf--p graf--leading'),
                    ('h3', 'graf graf--h3 graf--startsWithDoubleQuote graf--leading graf--title'),
                    ('h3', 'graf graf--h3 graf--startsWithDoubleQuote graf-after--figure graf--trailing graf--title')]
    title = None
    for combi in combinations:
        title = card.find(combi[0], class_=combi[1])
        if title is not None:
            return title.text
    return title

def get_subtitle(card):
    # Different combination of classes possible for subtitles
    combinations = [('h4', 'graf graf--h4 graf-after--h3 graf--subtitle'),
                    ('h4', 'graf graf--h4 graf-after--h3 graf--trailing graf--subtitle'),
                    ('strong', 'markup--strong markup--p-strong'),
                    ('h4', 'graf graf--p graf-after--h3 graf--trailing'),
                    ('p', 'graf graf--p graf-after--h3 graf--trailing'),
                    ('blockquote', 'graf graf--pullquote graf-after--figure graf--trailing'),
                    ('p', 'graf graf--p graf-after--figure'),
                    ('blockquote', 'graf graf--blockquote graf-after--h3 graf--trailing'),
                    ('p', 'graf graf--p graf-after--figure graf--trailing'),
                    ('em', 'markup--em markup--p-em'),
                    ('p', 'graf graf--p graf-after--p graf--trailing')]
    subtitle = None
    for combi in combinations:
        subtitle = card.find(combi[0], class_=combi[1])
        if subtitle is not None:
            return subtitle.text
    return subtitle

def get_author(card):
    author = card.find('a', class_='ds-link ds-link--styleSubtle link link--darken link--accent u-accentColor--textNormal u-accentColor--textDarken')
    if author is not None:
        return author.text
    return author

def get_publication(card):
    pub = card.find('a', class_='ds-link ds-link--styleSubtle link--darken link--accent u-accentColor--textNormal')
    if pub is not None:
        return pub.text
    return pub

def get_read_time(card):
    time = card.find('span', class_='readingTime')
    if time is not None:
        time = time['title']
        return time.replace(' min read', '')
    return time

def get_claps(card):
    claps = card.find('button', class_='button button--chromeless u-baseColor--buttonNormal js-multirecommendCountButton u-disablePointerEvents')
    if claps is not None:
        claps = claps.text
        if 'K' in claps:
            try:
                return int(float(claps.replace('K', '')) * 1000)
            except:
                return None
        else:
            try:
                return int(claps)
            except:
                return None
    return claps

def is_comment(card):
    # Check if card is a story or comment
    comment = card.find('div', class_='u-fontSize14 u-marginTop10 u-marginBottom20 u-padding14 u-xs-padding12 u-borderRadius3 u-borderCardBackground u-borderLighterHover u-boxShadow1px4pxCardBorder')
    return comment is not None

def get_url(card):
    url = card.find('a', class_='')
    if url is not None:
        return url['href'].split('?')[0]
    return url

def get_skills(title, subtitle):
    context = title
    if subtitle is not None:
        context = context + ' ' + subtitle
    all_skills = extract_skills(context)
    keep_skills, _ = extract_ignore(all_skills)
    keep_skills.sort()
    if len(keep_skills) > 0:
        data_skills = extract_data_skills(keep_skills)
        if len(data_skills) > 0:
            return '; '.join(keep_skills), '; '.join(data_skills)
        return '; '.join(keep_skills), None
    return None, None

In [ ]:
tag = 'data-science'
start_date = dt.datetime(2021, 9, 30)
end_date = dt.datetime(2021, 10, 18)
current_date = start_date

for i in range((end_date - start_date).days):
    df = medium_scraper(tag, current_date)
    if i == 0:
        df.to_csv('database/medium_data_science.csv', index=False)
    else:
        df.to_csv('database/medium_data_science.csv', index=False, mode='a', header=False)
    current_date = current_date + dt.timedelta(days=1)
    time.sleep(3)

In [ ]:
tag = 'machine-learning'
current_date = start_date

for i in range((end_date - start_date).days):
    df = medium_scraper(tag, current_date)
    if i == 0:
        df.to_csv('database/medium_machine_learning.csv', index=False)
    else:
        df.to_csv('database/medium_machine_learning.csv', index=False, mode='a', header=False)
    current_date = current_date + dt.timedelta(days=1)
    time.sleep(3)

In [ ]:
tag = 'data-engineering'
current_date = start_date

for i in range((end_date - start_date).days):
    df = medium_scraper(tag, current_date)
    if i == 0:
        df.to_csv('database/medium_data_engineering.csv', index=False)
    else:
        df.to_csv('database/medium_data_engineering.csv', index=False, mode='a', header=False)
    current_date = current_date + dt.timedelta(days=1)
    time.sleep(3)

In [ ]:
df_med = pd.read_csv('database/medium_data_science.csv')
df_med = df_med.append(pd.read_csv('database/medium_machine_learning.csv'))
df_med = df_med.append(pd.read_csv('database/medium_data_engineering.csv'))
df_med = df_med.drop_duplicates(subset=['id'])
df_med['published_date'] = pd.to_datetime(df_med['published_date'], dayfirst=True)
df_med = df_med.sort_values(by=['published_date', 'id'])
df_med['url'] = df_med['url'].apply(lambda x: x.split('?')[0])
df_med = df_med.reset_index(drop=True)
df_med.head()

In [ ]:
df_med.to_csv('database/medium.csv', index=False)

In [5]:
start_date = dt.datetime(2022, 3, 9)
end_date = dt.datetime(2022, 3, 26)
current_date = start_date
for i in range((end_date - start_date).days):
    print(current_date)
    current_date = current_date + dt.timedelta(days=1)

2022-03-09 00:00:00
2022-03-10 00:00:00
2022-03-11 00:00:00
2022-03-12 00:00:00
2022-03-13 00:00:00
2022-03-14 00:00:00
2022-03-15 00:00:00
2022-03-16 00:00:00
2022-03-17 00:00:00
2022-03-18 00:00:00
2022-03-19 00:00:00
2022-03-20 00:00:00
2022-03-21 00:00:00
2022-03-22 00:00:00
2022-03-23 00:00:00
2022-03-24 00:00:00
2022-03-25 00:00:00


In [6]:
tags = ['data-science', 'machine-learning', 'data-engineering']
# start_date = dt.datetime(2021, 12, 14)
# end_date = dt.datetime(2021, 12, 29)
df_med = pd.DataFrame()

for tag in tags:
    current_date = start_date
    print(tag)
    for i in range((end_date - start_date).days):
        df_temp = medium_scraper(tag, current_date)
        df_med = df_med.append(df_temp)
        current_date = current_date + dt.timedelta(days=1)
        time.sleep(random.randint(1,3))

df_med.head()

2022-03-09 00:00:00
2022-03-09 00:00:00
2022-03-09 00:00:00


,id,title,subtitle,author,publication,published_date,read_time_mins,claps,url,skills,data_skills
0,5b74a24501b0,Best of Both Worlds: Automated and Dynamic SQL...,Bring automation to new heights…,Yi Li,Towards Data Science,2022-03-09,4,160,https://towardsdatascience.com/best-of-both-wo...,Automation; Python; Structured Query Language ...,Python Programming; SQL
1,8bf616204765,10 SQL Queries You Should Know as a Data Scien...,Learn the Most Used SQL Queries in 5 Minutes w...,Uğur Savcı,SelectFrom,2022-03-09,6,159,https://selectfrom.dev/10-sql-queriesyou-shoul...,Structured Query Language (SQL),SQL
2,422a201c6947,7 Important Factors That Affect a Data Scienti...,Factors that determine how much you can…,Chris Zita,Data Science A-Z,2022-03-09,6,119,https://medium.com/data-science-playbook/7-imp...,None,None
3,6c797580b3b1,Network and Interconnection in Python Maps,A hands-on tutorial using Python packages incl...,Himalaya Bir Shrestha,Towards Data Science,2022-03-09,8,131,https://towardsdatascience.com/network-and-int...,Network; Python,Python Programming
4,ae900e2ac55f,"Text Summarization, Part 2 — State Of the Art ...",None,Mohamed Bamouh,Besedo Engineering Blog,2022-03-09,9,104,https://medium.com/besedo-engineering/text-sum...,Dataset,None


In [9]:
df_med = df_med.sort_values(by=['published_date', 'id'])
df_med = df_med.drop_duplicates(subset=['id'])
df_med

,id,title,subtitle,author,publication,published_date,read_time_mins,claps,url,skills,data_skills
8,2481103aa190,Community-Driven Learning,Learning together is the best way to learn 🚀,Mert Bozkır,None,2022-03-09,3,160,https://medium.com/@mertbozkir/community-drive...,None,None
5,2bf2104ccdcc,How to apply an ML model to a large csv in Python,As the size of databases and computational…,Giovanni Valdata,None,2022-03-09,6,263,https://medium.com/@giovanni-valdata/how-to-ap...,Database; Python,Python Programming
2,422a201c6947,7 Important Factors That Affect a Data Scienti...,Factors that determine how much you can…,Chris Zita,Data Science A-Z,2022-03-09,6,119,https://medium.com/data-science-playbook/7-imp...,None,None
0,45af9ed47de9,Predicting Tesla Stocks (TSLA) using Python & ...,None,Muneeb Ahmad,Dev Genius,2022-03-09,7,103,https://blog.devgenius.io/predicting-tesla-sto...,Python,Python Programming
0,574b655f0bc6,7 Antifragile Principles for a Successful Data...,None,Iliana Iankoulova,Picnic Engineering,2022-03-09,12,176,https://blog.picnic.nl/7-antifragile-principle...,Data Warehouse,None
...,...,...,...,...,...,...,...,...,...,...,...
0,a4730ba8d0e4,Pandas Is Not Enough? A Comprehensive Guide To...,None,Eryk Lewinson,Towards Data Science,2022-03-25,21,407,https://towardsdatascience.com/pandas-is-not-e...,Data Wrangling; Pandas,None
4,a5e993061fc7,Performing Analysis of Meteorological Data,The Hypothesis,Abdullah Abdul Wahid,None,2022-03-25,6,215,https://medium.com/@abdullahw72/performing-ana...,Analysis,None
7,e615ef62eeb6,Data Security for Dummies in 5 mins,Ever heard of “Data breach”? Assuming the answ...,Tanishq Tanwar,None,2022-03-25,4,216,https://medium.com/@akito7011/data-security-fo...,Blogging; Data Security,None
3,f0350f7c5b6d,Introduction to Text Preprocessing with Python,A beginner’s guide on text preprocessing with ...,Cem Bıkmaz,Python in Plain English,2022-03-25,7,342,https://python.plainenglish.io/introduction-to...,Python,Python Programming


In [10]:
engine = create_engine(settings['skills_db2'])
# df_med = pd.read_csv('database/medium.csv')
df_med['published_date'] = pd.to_datetime(df_med['published_date'])
# df_med.to_sql('ContentMedium', engine, index=False, if_exists='replace')
df_med.to_sql('ContentMedium', engine, index=False, if_exists='append')
engine.dispose()

## KDnuggets

In [ ]:
df_kd = pd.read_csv('database/kdnuggets.csv')
df_kd['date'] = pd.to_datetime(df_kd['date'])
df_kd.head()

In [ ]:
df_kd.info()

In [ ]:
engine = create_engine(settings['skills_db2'])
df_kd.to_sql('ContentKDnuggets', engine, index=False, if_exists='replace')
engine.dispose()

In [ ]:
dt.datetime.now() - dt.timedelta(days=7)

In [ ]:
dt.datetime.now() - dt.timedelta(days=7)